In [ ]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
import random # type: ignore
from pprint import pprint # type: ignore
import plotly.express as px # type: ignore
from tabulate import tabulate # type: ignore
from sklearn.metrics import confusion_matrix # type: ignore
from sklearn.metrics import accuracy_score # type: ignore

### Descriptive analysis

In [ ]:
path = "https://raw.githubusercontent.com/depa-tto/Machine-Learning-Module-Antonio-De-Patto/main/mashroom_dataset.csv"

df = pd.read_csv(path, sep = ';')

df = df.rename(columns={"class": "label"})
temp_cols = df.columns.tolist()
new_cols = temp_cols[1:] + temp_cols[0:1]
df = df[new_cols]

df.head()

In [ ]:
df.info()

In [ ]:
na_counts = df.isnull().sum()

table = [[col, na_counts[col]] for col in na_counts.index]
col_names = ["Features", "NA"]

print(tabulate(table, headers=col_names))

In [ ]:
df = df.drop(['gill-spacing','stem-root', 'stem-surface', 'veil-type', 'veil-color', 'spore-print-color'], axis = 1)

In [ ]:
df['cap-shape'] = df['cap-shape'].map({'b':'bell', 'c':'conical', 'x':'convex', 'f':'flat', 's':'sunken', 'p':'spherical', 'o':'others'})
df['cap-surface'] = df['cap-surface'].map({'i':'fibrous', 'g':'grooves', 'y':'scaly', 's':'smooth', 'h':'shiny', 'l':'leathery', 'k':'silky', 't':'sticky', 'w':'wrinkled', 'e':'fleshy', 'd': 'dry'})
df['cap-color'] = df['cap-color'].map({'n':'brown', 'b':'buff', 'g':'gray', 'r':'green', 'p':'pink', 'u':'purple', 'e':'red', 'w':'white', 'y':'yellow', 'l':'blue', 'o':'orange', 'k':'black'})
df['does-bruise-or-bleed'] = df['does-bruise-or-bleed'].map({'t':'bruises-bleedin', 'f':'not-bruises-bleedin'})
df['gill-attachment'] = df['gill-attachment'].map({'a':'bell', 'x':'conical', 'd':'convex', 'e':'flat', 's':'sunken', 'p':'spherical', 'f':'none', '?':'none'})
df['gill-color'] = df['gill-color'].map({'n':'brown', 'b':'buff', 'g':'gray', 'r':'green', 'p':'pink', 'u':'purple', 'e':'red', 'w':'white', 'y':'yellow', 'l':'blue', 'o':'orange', 'k':'black', 'f':'none'})
df['stem-color'] = df['stem-color'].map({'n':'brown', 'b':'buff', 'g':'gray', 'r':'green', 'p':'pink', 'u':'purple', 'e':'red', 'w':'white', 'y':'yellow', 'l':'blue', 'o':'orange', 'k':'black', 'f':'none'})
df['has-ring'] = df['has-ring'].map({'t':'ring', 'f':'none'})
df['ring-type'] = df['ring-type'].map({'c':'cobwebby', 'e':'evanescent', 'r':'flaring', 'g':'grooved', 'l':'large', 'p':'pendant', 's':'sheathing', 'z':'zone', 'y':'scaly', 'm':'movable', 'f':'none', '?':'none'})
df['habitat'] = df['habitat'].map({'g':'grasses', 'l':'leaves', 'm':'meadows', 'p':'paths', 'h':'heaths', 'u':'urban', 'w':'waste', 'd':'woods'})
df['season'] = df['season'].map({'s':'spring', 'u':'summer', 'a':'autumn', 'w':'winter'})



In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(how = 'any')

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

In [ ]:
dis = px.pie(df, names='label', color='label', color_discrete_sequence=['#008066','#B2D966'], title='Data Distribution')

dis.update_traces(textfont_size=18)  
dis.update_layout(width=700, height=500, plot_bgcolor='white', paper_bgcolor='white',
                  legend=dict(x=0.8, y=1, traceorder='normal', orientation='v', title_font=dict(size=16), font=dict(size=16)))

dis.show()

In [ ]:
class_counts = df['label'].value_counts()

print(class_counts)

In [ ]:
df.info() 

In [ ]:
for column in df.select_dtypes(include=np.number).columns:
    
    fig = px.box(data_frame=df, x='label', color='label', y=column, color_discrete_sequence=['#B2D966', '#008066'], orientation='v')
    fig.update_layout(
        width=600,   
        height=400,  
        plot_bgcolor='white', 
        paper_bgcolor='white',
        title=f'Box plot of {column}', 
        xaxis_title='Label',
        yaxis_title=column,
        showlegend=False
    )
    fig.show()

In [ ]:
cormat = df.select_dtypes(include=np.number).corr()
round(cormat,2)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cormat, annot=True, cmap='summer', cbar_kws={'shrink': .8}, linewidths=0.5)
plt.show()

In [ ]:
sns.pairplot(data=df, hue = 'label', palette=['#B2D966', '#008066'])

In [ ]:
target_column = 'label' 

for column in df.drop(columns=[target_column]).select_dtypes(exclude=[np.number]).columns:

    crosstab = pd.crosstab(df[column], df[target_column]).reset_index() 

    crosstab['total'] = crosstab['e'] + crosstab['p'] 
    crosstab = crosstab.sort_values(by='total', ascending=False)

    crosstab_melted = crosstab.melt(id_vars=[column], value_vars=['e', 'p'], 
                                    var_name=target_column, value_name='Count')
    fig = px.bar(crosstab_melted, 
                 x=column, 
                 y='Count', 
                 color=target_column,
                 labels={column: column, 'Count': 'Count', target_column: 'Type'},
                 title=f'Frequencies of Edible and Poisonous mushroom for {column}',
                 color_discrete_map={'e': '#008066', 'p': '#B2D966'})

    fig.update_layout(
        width=550,
        height=450,
        plot_bgcolor='white',
        paper_bgcolor='white',
        barmode='stack',  
        xaxis_title=column,
        yaxis_title='Count',
        legend=dict(
            x=0.9,  
            y=0.9,     
            title='Type',
            traceorder='normal',
            orientation='v'
        )
    )

    fig.update_xaxes(categoryorder='total descending')

    fig.show()


In [ ]:
df['cap-diameter'] = round(df['cap-diameter'], 3)
df['stem-width'] = round(df['stem-width'], 3)
df['stem-height'] = round(df['stem-height'], 3)

In [ ]:
df['cap-diameter'] = pd.to_numeric(df['cap-diameter'], errors='coerce')
df['stem-width'] = pd.to_numeric(df['stem-width'], errors='coerce')
df['stem-height'] = pd.to_numeric(df['stem-height'], errors='coerce')


In [ ]:
print(df.dtypes)

### Train and test split

In [ ]:
def split_train_test(df, test_size):

    """
    Splits the given dataset into training and testing subsets.
    
    Parameters:
    - df: the input dataset to be split.
    - test_size: specifies the size of the test set. Can be either:
        - An integer: number of rows for the test set.
        - A float: proportion of the total number of rows to be used as the test set.
    
    Returns:
    - A tuple (train_df, test_df) where:
        - train_df: dataset containing the training data.
        - test_df: dataset containing the test data.
    """
    
    if isinstance(test_size, float): # if the test size is a number or if is a proportion(float)
        test_size = round(test_size * len(df)) # we have to compute the number of rows this proportion represents

    indices = df.index.tolist() # list of row indices from the dataset
    test_indices = random.sample(population = indices, k = test_size) # we want to pick at random a certain number of these indices from this list

    test_df = df.loc[test_indices] # we create the test daframe by just indexing the rows with test indices
    train_df = df.drop(test_indices) # and the training set in created by dropping rows with the test indices
    
    return train_df, test_df

In [ ]:
random.seed(0)
train_df, test_df = split_train_test(df, test_size = 0.20)

In [ ]:
data = train_df.values # we transfrom from a pandas df to a numpy 2d array to make everything faster

In [ ]:
len(data)

In [ ]:
len(test_df)

### Type of feature

In [ ]:
for column in df.columns:
    print(column, '-', len(df[column].unique()))

In [ ]:
def categorize_features(df):

    """
    Categorizes the features in the dataset into either 'categorical' or 'continuous'.
    
    Parameters:
    - df: The input dataset containing the features to be classified 
    
    Returns:
    - A list `feature_types` where each element corresponds to the type of a feature in the dataset.
    """
    
    feature_types = [] # we initialize an empty list to store the type of each feature
    n_unique_values_treshold = 15 # threshold for the number of unique values to determine if a feature is categorical
    
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique() # we get the unique values in the feature column
            example_value = unique_values[0] # we initially pick the first value from the unique values as a sample

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [ ]:
FEATURE_TYPES = categorize_features(df)

In [ ]:
FEATURE_TYPES

### Is a node pure ?

In [ ]:
def evaluate_purity(data):   

    """
    Evaluates whether the data is pure, meaning all examples in the data belong to the same class.
    
    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    
    Returns:
    - A boolean value:
      - True: if all the examples in the data belong to the same class (i.e., the data is pure).
      - False: if the examples belong to different classes (i.e., the data is impure).
    """

    label_column = data[:, -1]
    unique_classes = np.unique(label_column) # how many distinct classes are in this array ? we use the numpy function 'unique'

    if len(unique_classes) == 1: 
        return True # if there's only one unique class, the data is pure
    else:
        return False

In [ ]:
label_column = data[:, -1]
unique_classes = np.unique(label_column) 

unique_classes

In [ ]:
evaluate_purity(data) 

In [ ]:
evaluate_purity(train_df[train_df.label == 'e'].values) 

In [ ]:
evaluate_purity(train_df[train_df['cap-diameter'] > 18].values)

### Classification

In [ ]:
def determine_majority_class(data):

    """
    Determines the majority class in the given data, so in this case we want to classify if a data is poisoned or not
    
    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    
    Returns:
    - classification: the class that occurs most frequently in the label column.
    """
    
    label_column = data[:, -1]

    # we are going to find the unique classes and their respective counts in the label column
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts = True) 

    # we need to know the index of the largest value of the 'counts_unique_classes' array to see which is the class that appears most often
    # so we want to identify the index of the class with the highest count (most frequent class)
    index = counts_unique_classes.argmax() 
    classification = unique_classes[index]
    
    return classification

In [ ]:
label_column = data[:, -1]
unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

In [ ]:
unique_classes, counts_unique_classes # in this case we can see that the p class is the one that appears most

In [ ]:
index = counts_unique_classes.argmax() 
classification = unique_classes[index]

In [ ]:
index

In [ ]:
classification # so p is the label that appears most often, and it is indexed 1, so it is in position 1 in the 'counts_unique_classes' array

In [ ]:
determine_majority_class(train_df[train_df['cap-diameter'] < 10].values) # so considering 'cap-diameter' lower than 10 the p category is the one that appear most 

### Potential split

In [ ]:
def find_possible_splits(data): # data which is a 2d numpy array

    """
    Identifies all possible splits for each feature in the dataset.

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.

    Returns:
    - potential_splits: a dictionary where the keys are column indices (features), and the values are arrays of possible split points.
    """
    
    potential_splits = {} # dictionary, that has as keys the indeces of the columns and as values the list that contains all the potential splits
    _, n_columns = data.shape # tuple that return the number of rows and the number of columns that we have in the dataframe, we only take the number of columns
    for column_index in range(n_columns - 1): # excluding the last column which is the label
        values = data[:, column_index] # values from the current feature column
        unique_values = np.unique(values) # all unique values in this column
        
        potential_splits[column_index] = unique_values # for every columns we are going to create an entry in our potential split dictionary, so we are going to append our potential split
    
    return potential_splits

In [ ]:
data.shape

In [ ]:
find_possible_splits(data) # the keys are the indecis of the columns and the values are the rows that contain numbers for the potential split, that are the unique values for every columns

In [ ]:
potential_split = find_possible_splits(data)

sns.lmplot(data = train_df, x = 'cap-diameter', y = 'stem-width', hue = 'label', fit_reg = False)

# plt.vlines(x = potential_split[0], ymin = 0, ymax = 100)
# plt.hlines(y = potential_split[8], xmin = 0, xmax = 60)

In [ ]:
sns.lmplot(data = train_df, x = 'cap-diameter', y = 'stem-height', hue = 'label', fit_reg = False)

# plt.vlines(x = potential_split[0], ymin = 0, ymax = 100)
# plt.hlines(y = potential_split[8], xmin = 0, xmax = 60)

In [ ]:
sns.lmplot(data = train_df, x = 'cap-diameter', y = 'stem-width', hue = 'label', fit_reg = False)

# plt.vlines(x = potential_split[0], ymin = 0, ymax = 100)
# plt.hlines(y = potential_split[8], xmin = 0, xmax = 60)

### Split data

In [ ]:
def split_data(data, split_column, split_value): 

    """
    Splits the data into two subsets based on a specified feature and split value.

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    - split_column: the index of the column (feature) on which to split the data.
    - split_value: the value in the feature column to use as the threshold for splitting.

    Returns:
    - data_below: subset of data where the feature values are less than or equal to the split value (for continuous features) or equal to the split value (for categorical features).
    - data_above: subset of data where the feature values are greater than the split value (for continuous features) or not equal to the split value (for categorical features).
    """
    
    split_column_values = data[:, split_column]  # we extract all the values from the specified column

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous": # if the feature is continuous, split based on the threshold value
        data_below = data[split_column_values <= split_value] # data below the split value
        data_above = data[split_column_values >  split_value] # data above the split value
    
    # feature is categorical   
    else: # if the feature is categorical, split based on matching the value
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [ ]:
# example continuos variable

split_column = 0
split_value = 5

split_column_values = data[:, split_column]

split_column_values <= split_value

In [ ]:
# example categorical variables 

split_column = 1
split_value == 'b'

split_column_values = data[:, split_column]

split_column_values == split_value

### Scaled entropy

In [ ]:
def scaled_entropy(data): 

    """
    Computes the scaled entropy of a dataset

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.

    Returns:
    - entropy: the scaled entropy value for the dataset.
    """
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts = True) # unique classes and their counts in the label column

    probabilities = counts / counts.sum() # probability of each class
    entropy = sum((- probabilities/2) * np.log2(probabilities + 1e-10) - (1 - probabilities)/2 * np.log2(1 + 1e-10 - probabilities))
     
    return entropy

In [ ]:
_, counts = np.unique(label_column, return_counts=True)
print(counts)

In [ ]:
counts.sum()

In [ ]:
probabilities = counts / counts.sum()

probabilities

In [ ]:
scaled_entropy(data) # near to one, so there is a high confunsion between the data

In [ ]:
scaled_entropy(test_df.values)

In [ ]:
def calculate_overall_entropy(data_below, data_above): # we will compute the entropy belonging to the below data and to the above data

    """
    Computes the weighted average of the entropy for two subsets of data (data_below and data_above).
    This is used to evaluate the quality of a split in a decision tree.

    Parameters:
    - data_below: a 2D numpy array representing the subset of data that falls below a certain split value.
    - data_above: a 2D numpy array representing the subset of data that falls above a certain split value.

    Returns:
    - overall_entropy: the weighted entropy of the combined subsets.
    """
    
    n = len(data_below) + len(data_above) # total number of data points in both subsets
    p_data_below = len(data_below) / n # proportion of data points in the 'data_below' subset
    p_data_above = len(data_above) / n # proportion of data points in the 'data_above' subset

    # we compute the overall weighted entropy by taking the weighted sum of the entropies of both subsets
    overall_entropy =  (p_data_below * scaled_entropy(data_below) 
                      + p_data_above * scaled_entropy(data_above))
    
    return overall_entropy

### Gini function

In [ ]:
def gini_impurity (data):

    """
    Computes the Gini impurity of a dataset

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.

    Returns:
    - gini: the Gini impurity of the dataset.
    """
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    gini = sum(2*probabilities * (1 - probabilities))
     
    return gini

In [ ]:
gini_impurity (data)

In [ ]:
def calculate_overall_gini(data_below, data_above):

    """
    Computes the weighted Gini impurity for a given split of data.

    Parameters:
    - data_below: a subset of the data after applying a split, containing instances that fall below the split value.
    - data_above: a subset of the data after applying a split, containing instances that fall above the split value.

    Returns:
    - overall_gini: the weighted Gini impurity of the split.
    """
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_gini =  (p_data_below * gini_impurity (data_below) 
                    + p_data_above * gini_impurity (data_above))
    
    return overall_gini

### Bhattacharyya coefficient

In [ ]:
def bhattacharyya_coefficient (data):

    """
    Computes the Bhattacharyya Coefficient of a dataset

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.

    Returns:
    - bhatt_coeff: the Bhattacharyya Coefficient of the dataset.
    """
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    bhatt_coeff = sum(np.sqrt(probabilities * (1 - probabilities)))

    return bhatt_coeff

In [ ]:
bhattacharyya_coefficient (data)

In [ ]:
def calculate_overall_bhattacharyya_coefficient (data_below, data_above):

    """
    Computes the weighted Bhattacharyya coefficient for a given split of data.

    Parameters:
    - data_below: a subset of the data after applying a split, containing instances that fall below the split value.
    - data_above: a subset of the data after applying a split, containing instances that fall above the split value.

    Returns:
    - overall_bhatt_coeff: the weighted Bhattacharyya coefficient of the split.
    """
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_bhatt_coeff = (p_data_below * bhattacharyya_coefficient (data_below) 
                      + p_data_above * bhattacharyya_coefficient (data_above))
    
    return overall_bhatt_coeff

### Determine best split according to the Scaled Entropy

In [ ]:
# we would like to look at all the potential split and determine the one split that result in the lowest overall entropy 

def determine_best_split_scaled_entropy(data, potential_splits):

    """
    Determines the best split for the dataset based on the scaled entropy criterion.
    This function should look at all the potential split and then determine the one split that result in the lowest overall entropy

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    - potential_splits: a dictionary where keys are column indices and values are lists of potential split values for those columns.

    Returns:
    - best_split_column: the index of the column where the best split occurs.
    - best_split_value: the value of the feature where the best split occurs.
    """
    
    overall_entropy = 9999  # we initialize the overall entropy with a very high value
    for column_index in potential_splits: # so this will loop over the keys that are the column indices
        for value in potential_splits[column_index]: # this will loop over all the elements of the dictionary
            data_below, data_above = split_data(data, split_column=column_index, split_value=value) # split the data based on the current feature and split value
            current_overall_entropy = calculate_overall_entropy(data_below, data_above) # calculate the overall entropy for the current split

            if current_overall_entropy <= overall_entropy: # is the current overall entropy smaller or equal than the initial overall entropy ? it can be equal because there could be more split that gives back the same cut in the entropy
                overall_entropy = current_overall_entropy # if yes we are going to update the overall entropy and we are going to save this values in the 'best_split_column' and 'best_split_value' values
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

### Determine best split according to the Gini Impurity

In [ ]:
def determine_best_split_gini(data, potential_splits):

    """
    Determines the best split for the dataset based on the Gini impurity criterion.
    This function should look at all the potential split and then determine the one split that result in the lowest overall gini impurity

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    - potential_splits: a dictionary where keys are column indices and values are lists of potential split values for those columns.

    Returns:
    - best_split_column: the index of the column where the best split occurs.
    - best_split_value: the value of the feature where the best split occurs.
    """
    
    overall_gini = 9999
    for column_index in potential_splits: 
        for value in potential_splits[column_index]: 
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_gini = calculate_overall_gini(data_below, data_above)

            if current_overall_gini <= overall_gini:
                overall_gini = current_overall_gini 
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

### Determine best split according to the Bhattacharyya coefficient

In [ ]:
def determine_best_split_bhattacharyya_coefficient(data, potential_splits):

    """
    Determines the best split for the dataset based on the Bhattacharyya coefficient criterion.
    This function should look at all the potential split and then determine the one split that result in the lowest overall Bhattacharyya coefficient

    Parameters:
    - data: a 2D numpy array of the training set where each row is an example and the last column contains the labels.
    - potential_splits: a dictionary where keys are column indices and values are lists of potential split values for those columns.

    Returns:
    - best_split_column: the index of the column where the best split occurs.
    - best_split_value: the value of the feature where the best split occurs.
    """
    
    overall_3 = 9999
    for column_index in potential_splits: 
        for value in potential_splits[column_index]: 
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_3 = calculate_overall_bhattacharyya_coefficient (data_below, data_above)

            if current_overall_3 <= overall_3:
                overall_3 = current_overall_3 
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [ ]:
def split_method(data, potential_splits, criterion='gini'):
    if criterion == 'scaled_entropy':
        return determine_best_split_scaled_entropy(data, potential_splits)
    elif criterion == 'gini':
        return determine_best_split_gini(data, potential_splits)
    elif criterion == 'bhatt_coeff':
        return determine_best_split_bhattacharyya_coefficient(data, potential_splits)

### Decision Tree Algorithm

In [ ]:
def decision_tree(df, counter = 0, min_samples = 200, max_depth = 5, criterion = 'scaled_entropy'):

    """
    Constructs a decision tree for the given dataset.

    Parameters:
    - df: the input dataframe where rows are samples and columns include features and the label.
    - counter: the current depth of the tree (used for recursion).
    - min_samples: minimum number of samples required to split an internal node.
    - max_depth: maximum depth of the tree.
    - criterion: the criterion used to choose the best split ('scaled_entropy', 'gini', 'bhatt_coeff').

    Returns:
    - A decision tree 
    """
    
    # data preparations for the first function call. we initialize global variables for column headers and feature types
    if counter == 0: # so in the first call of the function we give a general information about the data since all the helper function works for a 2d numpy array 
        global COLUMN_HEADERS, FEATURE_TYPES # we specify these variables as globals
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = categorize_features(df)
        data = df.values
    else:
        data = df # for recursive calls, use the provided data          
    
    
    # base cases, where the stopping conditions are presented
    # evaluate_purity gives back a boolean array that can be directly be classifies
    # also we classify a data if there are not 'min_samples' datapoints, even though it could not be pure yet
    # so if in a particular node the number of samples becomes less than the minimum samples then we will not split that node any further and it will be a leaf node
    # if the depth of the tree reach the maximum depth we will not split the nodes further
    if (evaluate_purity(data)) or (len(data) < min_samples) or (counter == max_depth): 
        classification = determine_majority_class(data)
        
        return classification

    
    # recursive part:split the data and continue building the tree
    else:    
        counter += 1

        potential_splits = find_possible_splits(data) # we determine possible splits for each feature
        split_column, split_value = split_method(data, potential_splits, criterion=criterion) # the we select the best split based on the chosen criterion
        data_below, data_above = split_data(data, split_column, split_value) # and finally we split the data into two subsets based on the best split
        
        # check for case where a split results in empty subsets
        if len(data_below) == 0 or len(data_above) == 0: 
            classification = determine_majority_class(data)
            return classification
        
        # determine question for the decision tree node
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # creation of a new sub-tree with the formulated question
        sub_tree = {question: []} # in that empty list we want to append the yes or no answer 
        
        # recursively build the subtrees for the 'yes' and 'no' branches of the current split
        yes_answer = decision_tree(data_below, counter, min_samples, max_depth, criterion)
        no_answer = decision_tree(data_above, counter, min_samples, max_depth, criterion)
        
        # if the answers are the same, then there is no point in asking the question and we can use that answer directly
        # this could happen when the data is classified even though it is not pure yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

### Evaluation functions

In [ ]:
def predict_example(example, tree):

    """
    Predicts the class label for a given example using the provided decision tree.

    Parameters:
    - example: a dictionary representing a single data instance with feature names as keys.
    - tree: the decision tree previously computed.

    Returns:
    - The predicted class label for the example.
    """

    question = list(tree.keys())[0] # get the question at the current node of the tree
    feature_name, comparison_operator, value = question.split(" ") # for example feature name is 'stem-width', comparison operator is '<=' and value is '8.85'

    # ask the question and determine which branch to follow
    
    # if feature is continuous, compare the feature value to the split value
    if comparison_operator == "<=":  
        if example[feature_name] <= float(value):
            answer = tree[question][0] # yes answer
        else:
            answer = tree[question][1] # no answer
    
    # if the feature is categorical, compare the feature value to the split value
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # if the answer is not a dictionary, then it is a leaf node, and we return the predicted class label
    if not isinstance(answer, dict):
        return answer
    
    # recursive part: if the answer is a dictionary, continue traversing the tree 
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

In [ ]:
def compute_accuracy(df, tree):

    """
    Computes the accuracy of a decision tree classifier on a given dataset.

    Parameters:
    - df: the input dataset 
    - tree: the decision tree previously computed

    Returns:
    - The accuracy of the decision tree on the given dataset.
    """
    
    # we have to apply the prediction function to each row of the dataset
    # The 'args' parameter is used to pass additional arguments to the function (i.e., the decision tree).
    # 'axis=1' ensures that the function is applied row-wise.
    df["classification"] = df.apply(predict_example, args = (tree,), axis = 1) # this variable will contain the classfification of our examples

    # new column to check if the classification is correct. It contains a boolean value (True/False) indicating whether the predicted class matches the true class.
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean() # mean of the 'classification_correct' column, which gives the proportion of correct classifications.
    
    return accuracy

In [ ]:
def zero_one_loss(y_true, y_pred):

    """
    Computes the zero-one loss between true labels and predicted labels.

    Parameters:
    - y_true: a list or array-like object containing the true class labels.
    - y_pred: a list or array-like object containing the predicted class labels.

    Returns:
    - The zero-one loss, which represents the proportion of incorrect predictions.
    """

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)


    # element-wise comparison between true labels and predicted labels
    # np.sum counts the number of False values (which indicate incorrect predictions)
    
    incorrect_predictions = np.sum(y_true != y_pred)
    
    loss = incorrect_predictions / len(y_true) #  proportion of incorrect predictions relative to the total number of samples
    
    return loss

### Tree output

In [ ]:
random.seed(0)
train_df, test_df = split_train_test(df, test_size = 0.2)

"""
Parameters:
    - min_samples: minimum number of samples required to split an internal node.
    - max_depth: maximum depth of the tree.
    - criterion: the criterion used to choose the best split ('scaled_entropy', 'gini', 'bhatt_coeff').

"""

tree = decision_tree(train_df, min_samples = 500, max_depth = 5, criterion='scaled_entropy')
pprint(tree)

### Classification of examples

In [ ]:
example = test_df.iloc[0]
example

In [ ]:
list(tree.keys())[0]

In [ ]:
example['stem-width']

In [ ]:
example['stem-width'] <= 8.85

In [ ]:
question = list(tree.keys())[0]
tree[question][1] # no answer

In [ ]:
predict_example(example, tree)

### Tree evaluation

In [ ]:
y_pred_train = [predict_example(train_df.loc[i], tree) for i in train_df.index]

y_pred_test = [predict_example(test_df.loc[i], tree) for i in test_df.index]

In [ ]:
y_true_train = train_df['label']

In [ ]:
y_true_test = test_df['label']

In [ ]:
zero_one_loss(y_true_test, y_pred_test)

In [ ]:
zero_one_loss(y_true_train, y_pred_train)

In [ ]:
compute_accuracy(test_df, tree)

In [ ]:
compute_accuracy(train_df, tree)

In [ ]:
confusion_matrix(y_true_test, y_pred_test)

In [ ]:
classes = ['edible', 'poisoned']
features = df.columns

sns.heatmap(confusion_matrix(y_true_test, y_pred_test),yticklabels=classes,
            xticklabels=classes,annot=True,cmap='summer', fmt='g')
plt.tight_layout()
plt.show()

In [ ]:
test_df

In [ ]:
test_df.loc[38308]

In [ ]:
test_df.loc[49293]

In [ ]:
test_df = test_df.iloc[:, :-2]

In [ ]:
train_df = train_df.iloc[:, :-2]

### Hyperparameter tuning

In [ ]:
grid_search = {"max_depth": [], "min_samples": [], "accuracy_train": [], "accuracy_test": []}

for max_depth in range(3, 7):
    for min_samples in range(500, 3000, 500):

        tree = decision_tree(train_df, max_depth=max_depth, min_samples=min_samples, criterion = 'scaled_entropy')

        y_true_train = train_df['label'].values
        y_true_test = test_df['label'].values

        y_pred_train = [predict_example(train_df.loc[i], tree) for i in train_df.index]
        y_pred_test = [predict_example(test_df.loc[i], tree) for i in test_df.index]


        accuracy_train = accuracy_score(y_true_train, y_pred_train)
        accuracy_test = accuracy_score(y_true_test, y_pred_test)

        grid_search["max_depth"].append(max_depth)
        grid_search["min_samples"].append(min_samples)
        grid_search["accuracy_train"].append(accuracy_train)
        grid_search["accuracy_test"].append(accuracy_test)
        
    print(f"Progress: Iteration max_depth={max_depth}/6")

grid_search_df = pd.DataFrame(grid_search)
grid_search_df.sort_values("accuracy_test", ascending=False)


In [ ]:
train_accuracies = []
test_accuracies = []


for max_depth in range(1, 25):
    
    tree = decision_tree(train_df, min_samples=500, max_depth=max_depth, criterion = 'scaled_entropy')

    y_true_train = train_df['label'].values

    
    y_true_test = test_df['label'].values

    y_pred_train = [predict_example(train_df.loc[i], tree) for i in train_df.index]
    y_pred_test = [predict_example(test_df.loc[i], tree) for i in test_df.index]


    accuracy_train = accuracy_score(y_true_train, y_pred_train)
    accuracy_test = accuracy_score(y_true_test, y_pred_test)

    train_accuracies.append(accuracy_train)
    test_accuracies.append(accuracy_test)

In [ ]:
plt.figure(figsize = (10, 5))
sns.set_style("whitegrid")
plt.plot(train_accuracies, label= "train accuracy")
plt.plot(test_accuracies, label="test accuracy")
plt.legend(loc = "upper left")
plt.xticks(range(0, 26, 5))
plt.xlabel("max_depth", size = 15)
plt.ylabel("accuracy", size = 15)
plt.show()

In [ ]:
random.seed(0)

train_df, test_df = split_train_test(df, test_size=0.2)
tree = decision_tree(train_df, min_samples = 20, max_depth=5)
accuracy = compute_accuracy(test_df, tree)

pprint(tree)
accuracy

### K-fold cross validation

In [ ]:
df = df.sample(frac=1, random_state=0).reset_index(drop=True)

In [ ]:
def kfold_indices(df, k):
    fold_size = len(df) // k
    indices = np.arange(len(df))
    folds = []
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])
        folds.append((train_indices, test_indices))
    return folds

k = 5

fold_indices = kfold_indices(df, k)

In [ ]:
fold_indices

In [ ]:
cnt = 1

for train_indices, test_indices in fold_indices:
    print(f'Fold:{cnt}, Train set: {len(train_indices)}, Test set:{len(test_indices)}')
    cnt += 1
    print(test_indices)

In [ ]:
scores = []

for train_indices, test_indices in fold_indices:
    train_df = df.iloc[train_indices]
    test_df = df.iloc[test_indices]

    tree_cv = decision_tree(train_df, min_samples=500, max_depth=5, criterion='scaled_entropy')

    y_true_test = test_df['label'].values
    y_pred_test = [predict_example(test_df.loc[i], tree_cv) for i in test_df.index]

    fold_score = accuracy_score(y_true_test, y_pred_test)

    scores.append(fold_score)

mean_accuracy = np.mean(scores)
print("K-Fold Cross-Validation Scores:", scores)
print("Mean Accuracy:", mean_accuracy)